## 因子汇总

### 买卖单因子

* buy_ls_ratio_ret  

$ (BuyL_i / BuyS_i) * Ret_i$  
大小买单比率乘以收益率 某种程度上想刻画主力和散户的获利差距  
分组清晰均匀稳定 ir=0.5

* buy_per_ret_log  

$ log(\frac{B_i}{B_{i-1}} * Ret_i)$   

* exce_buy_l_log  

$ log((B_i - B_{i-1}\frac{Vol_i}{Vol_{i-1}})) $  

* netbuy_per_log_demed  

$f = log(\frac{B_i - S_i}{B_{i-1} - S_{i-1}})$  
$f=|(f - f.median())| / f.std() $   
减去中位数取绝对值是想改变因子的分布 下面同理

* buy_ret_act_trans   

$ f = BAct_i * Ret_i$  
$ f = |(f-f.median())| / f.std()$

- bsratio_morn30per_prod_log  

$ bsratio = B_i / S_I$  
$ morn30 = \text{开盘30min成交量} / \text{当日成交量}$  
$ f = np.log(bsratio * morn30)$


- buy_l_order_act_mktovermean  

$mktovermean = 1\; if\; mkt_{stock} > mkt_mean \;else\; NaN$    
$f = BAct * mktovermean$  
mkt：某日的市值取对数；

- buy_l_act_r10_mean_std_ratio  

$ BuyLAct$主动买入大单量，日频，取对数，后仍记作$BuyLAct$
$factor = BuyLAct.rolling(10).mean() / BuyLAct.rolling(10).std()$  
或者分子直接使用BuyLAct

- buy_l_order_act_demean  

$factor = BuyLAct / BuyLAct.rolling(10).mean()$  
很简单的形式但是还挺好

- buy_mean_devide_std_r20  

$BuyL.rolling(20).mean() / BuyL.rolling(20).std()$  
或者把mean换成max

- buy_ls_per_ratio_ret_log  



- buy_ls_ratio_ret

$(BuyL / BuyS)*Ret$

- sell_l_ret_log  

$log(SellL * Ret)$    
小于0的值直接设为NaN,空值虽然多，但是效果挺显著，也可以直接先取对数再乘Ret,有效减少空值数，效果还不错  
log(SellL * -Ret)效果也不错  

- exce_buy_abs_log

$ExceBuy = Buy_i - Buy_{i-1} * \frac{Vol_i}{Vol_{i-1}}$  
$factor = log(abs(ExceBuy))$

- buy_sell_square_sum_r10mean  

$ (BuyL^2 + Sell^2).rolling(10).sum()$

- buy_l_s_ratio_r10mean  

$ (BuyL/BuyS).rolling(10).mean()$

- buy_vol_log_r10_ols_slope

取$BuyL$和 $Vol$ rolling10日的数据，返回线性回归的斜率

### 价量

- vol_ret_over1std_per  

分钟频数据， 每日分钟频收益率偏离1sigma的成交量占比  
$Ret_{s,T,t}$表示单只股票s在日期T内,t分钟的收益率,用分钟的收盘价, $Vol_{s, T, t}$同理, 并记Vol_{s, T}为其向量  
    
将$Ret_{s,T,t}$按照分钟标准化，得到的序列仍记做$Ret_{s,T,t}$  
I = $Ret_{s,T,t}$.apply(lambda x: 1 if abs(x) > 1 else 0)  
$factor_{sT} = (I * Vol_{s, T}).sum() / Vol_{s, T}.sum()$向量点乘




- p_log_vol_dis_std_r10mean  

成交量在价格上的分布  
stockdaily为某只股票单日分钟级价格成交量数据  
stockdaily = stockdaily.set_index('price')['vol']  
fa = log(stockdaily).std()

- morn_afno_30_ret_r20corr  

取20天的早盘30分钟的收益率和尾盘30分钟的收益率做corr

- afno30_ret_cvar_neg_09_r20.csv  

$Afno30Ret:$尾盘30min收益率  
$factor = Afno30Ret.rolling(20)$中选取最小的两个值求平均

- volmin_log_zs_skew  

分钟级别的成交量数据, 取对数(记log(0)=NaN)，对单只股票做zscore之后返回偏度

- volmin_log_zs_morn30_devide_tail30_mean

```python
def f(x):
    x = ((x.T - x.mean(axis=1)) / x.std(axis=1)).T
    return x[x > 1].mean() / (x[x<-1].mean() + 1e-5)
volmin_log_zs_oer1std_devide_lower1std =\
     volmin_log.groupby(level='Date').progress_apply(lambda x: f(x))

- below_pcloday_vol_per

```python
pret = pmin / pmin.iloc[-1]
pret[pret > 1] = 0 #只计算低于收盘价的时刻
volmin = np.log(volmin)
factor = np.dot(volmin, pret) / volmin.sum()
factor = factor.rolling(5).mean()

计算在小于收盘价买入的力量

- last10ret_last10vol_per  
在9:30 1m_vwap上表现显著

last10ret=尾盘10分钟收益率  
last10vol_per:尾盘10分钟成交量在整天成交量的占比, 分钟成交量数据取对数  
factor = last10ret * last10vol_per

- volmin_fft_max10_mean_log
分钟成交量 快速傅里叶变换 取强度前十的信号做均值,最后取对数调整分布

```python
def handle_stockly_daily(data_stock_daily:pd.Series):
        vol = data_stock_daily['LastVolume']
        vol = vol.iloc[:-3] #去掉尾盘竞价
        N = 237
        fft_res = np.fft.fft(vol)
        fft_freq = np.fft.fftfreq(N, 1/N)
        res = pd.Series(np.abs(fft_res)[:N//2], index=fft_freq[:N//2])
        res = res.sort_values(ascending=False)

        return res.iloc[:10].mean()

- volmin_std_fft_over_mean1std_inv_std_ratio

vol为分钟频成交量去掉前30min和最后3min  
vol做Fourier变换后，保留振幅强度前十的信号再做逆变换得到vol_new  
factor = vol.std() / vol_new.std()
```python
def handle_stockly_daily(data_stock_daily:pd.Series):
        vol = data_stock_daily.iloc[30:-3]
        N = len(vol)
        vol.index = list(range(N))
        yf = pd.Series(np.fft.fft(vol))
        fft_freq = np.fft.fftfreq(N, 1/N)

        res = pd.Series(np.abs(yf)[:N//2], index=fft_freq[:N//2])
        res = res.sort_values(ascending=False)
        res_drop0 = res.iloc[1:].sort_values(ascending=False)
        idx = res_drop0.index[:10]

        yf2 = pd.Series(np.zeros(len(yf)))
        yf2.loc[idx] = yf.loc[idx]
        vol_new = pd.Series(np.real(np.fft.ifft(yf2)))

        return vol.std() / (vol_new.std() + 1e-5)

return data_stock.groupby(level='Date').apply(handle_stockly_daily)

- openret_fft_corr_logvol_r20std  

后来事实证明，好像不做fft直接算结果也是一样的...  
日内分钟频数据
$Openret = p / p.iloc[0]$  
$RetRft:$将openret做fft后保留振幅前十的信号，再做逆变换  
$LogVol:$分钟级成交量取对数  
$factor = corr(RetFft, LogVol).rolling(20).mean()$
```python
def handle_stockly_daily(data_stock_daily:pd.Series):
    p = data_stock_daily['Close']
    vol = np.log(data_stock_daily['LastVolume'].replace(0, np.nan))
    vol.index = list(range(len(vol)))
    ret = p / p.iloc[0]

    N = len(ret)
    ret.index = list(range(N))

    yf = pd.Series(np.fft.fft(ret))
    fft_freq = np.fft.fftfreq(N, 1/N)

    res = pd.Series(np.abs(yf)[:N//2], index=fft_freq[:N//2])
    res = res.sort_values(ascending=False)
    num = 10
    res_drop0 = res.iloc[1:].sort_values(ascending=False)
    idx = res_drop0.index[:num]

    yf2 = pd.Series(np.zeros(len(yf)))
    yf2.loc[idx] = yf.loc[idx]
    ret_new = pd.Series(np.real(np.fft.ifft(yf2)))

    return ret_new.corr(vol)


- min_p_vol_volfirst_win10_powersum_r20std

m = array(-5, -4, ..., 3, 4)  
res = [(vol.shift(i).corr(p)) * i  for i in m]  
corr_powersum = res.sum()
factor = corr_powersum.rolling(20).std()

```python
def handle_stockly(data_stock: pd.DataFrame):
    def handle_stockly_daily(data_stock_daily:pd.Series):
        m = 10
        m_array = np.arange(m) - m//2

        p = data_stock_daily['Close']
        vol = np.log(data_stock_daily['LastVolume'].replace(0, np.nan))

        res = 0
        for i in m_array:
            res += vol.shift(i).corr(p) * i

        return res
    
    return data_stock.groupby(level='Date').apply(handle_stockly_daily)
计算的结果再rolling(20).std()

### 非对称性因子